In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import math

# Task describtion

Завдання цього [змагання](https://www.kaggle.com/c/DontGetKicked/overview) полягає в тому, щоб передбачити, чи автомобіль, придбаний на аукціоні, є проблемним(BadBuy): скручений одометр, був у дтп, прихований номер або VIN-код, відстуня частина документів і т.д.

Tasklist:

1. Провести EDA
    1. Прочитати опис колонок
    2. Подивитись загальні дані про датасети
        * розмір датасету
        * типи данних
        * дублікати
        * missing values
        * розподіли
        * статистичні дані числових колонок
        * статистичні дані категоріальних колонок
        * оцінити кореляцію колонок
2. Оцінити відмінності між train та test датасетами
3. Підготовити дані до тренування
    1. Визначити які колонки являються невалідними для задачі на основі:
        * знань доменної області та опису колонок
        * статистичних данних
        * кореляції
        * кількості missing values
    2. Видилити непотрібні колонки та дублікати
    3. Обробити missing values
        * для надто великої кількості missing values у випадку категоріальної колонки утоврити нову категорію
        * якщо рядків з mv мало, можна їх видилити
        * заповнити модою та середнім залишок mv у категоріальних та чисельних колонках відповідно
    4. За необхідності збалансувати данні
    5. Провести стандартизацію данних(наприклад за допомогою sklearn.preprocessing.StandardScaler)
4. Тренування моделі
    1. Обрати модель
    2. Розділити train дані на train та validation датасети
    3. Обрати метрику
    4. Провести валідацю
    5. Прогонка моделі на тестових данних
5. Сабміт результату

In [ ]:
train = pd.read_csv('../input/DontGetKicked/training.csv')
test = pd.read_csv('../input/DontGetKicked/test.csv')

# EDA

## Common Info

Продивились кількість рядків/колонок у трейн/тест датасетах

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
train.dtypes

Визначаємо тип колонок
```python
categorical_features
```
та таргет-колонку "IsBadBuy" як категоріальні. Висновок по тому яка колонка є категоріальною, а яка ні робимо згідно їх опису в ../input/DontGetKicked/Carvana_Data_Dictionary.txt

In [ ]:
categorical_features = { "Auction", "Make", "Model", "PurchDate", "Size", "Color",
                        "Trim", "SubModel", "Transmission", "WheelType", "WheelTypeID",
                        "Nationality", "TopThreeAmericanName", "PRIMEUNIT",
                        "AUCGUART", "VNZIP1", "VNST", "IsOnlineSale"
                        }
train["IsBadBuy"] = train["IsBadBuy"].astype("category")
for feature in categorical_features:
    train[feature] = train[feature].astype("category")
    test[feature] = test[feature].astype("category")

In [ ]:
train.dtypes

## Missing Data

In [ ]:
print(train.isnull().sum())

In [ ]:
print(test.isnull().sum())

# Duplicates

In [ ]:
train[train.duplicated()]

In [ ]:
test[test.duplicated()]

In [ ]:
numerical_features = train.select_dtypes(include = ['float64', 'int64']).columns
train[numerical_features].hist(figsize=(20, 40), color = 'g', bins=30, xlabelsize=8, ylabelsize=8)

In [ ]:
train["IsBadBuy"].describe()

In [ ]:
train[categorical_features].describe()

In [ ]:
test[categorical_features].describe()

# Correlation Matrix

In [ ]:
corrMatt = train.corr()
mask = np.array(corrMatt)
mask[np.tril_indices_from(mask)] = False
fig,ax= plt.subplots()
fig.set_size_inches(120,60)
sns.heatmap(corrMatt, cmap="Greens", mask=mask,vmax=.8, square=True,annot=True)

In [ ]:
corrMatt

# Data prep

## Delete columns

WheelTypeID выдаляємо, бо в нас вже є аналогічна категоріальна колонка. PurchDate - не валідна для моделі.


In [ ]:
train.drop("WheelTypeID", inplace=True, axis=1)
test.drop("WheelTypeID", inplace=True, axis=1)

train.drop("PurchDate", inplace=True, axis=1)
test.drop("PurchDate", inplace=True, axis=1)

categorical_features.remove("PurchDate")
categorical_features.remove("WheelTypeID")

Видаляємо колонки з надто великою кількістю категорій. 
Спірна колонка - Color. З одного боку не типові кольори можут означати, що машині замніяли деталі та перекрашували, з іншого боку маємо занадто велику кількість категорій, все ж вирішили видалити.

In [ ]:
to_much_cat_delete_candidate = { "Model", "Trim", "SubModel", "VNZIP1", "VNST", "Make", "Color"}

for d in to_much_cat_delete_candidate:
    train.drop(d, inplace=True, axis=1)
    test.drop(d, inplace=True, axis=1)
    categorical_features.remove(d)

Видаляємо занадто сильно корельовані колонки, залишаємо дві колонки з інформацією про ціну, бо вважаємо що вони мають суттєвий вплив на таргет.

In [ ]:
corr_delete_candidate = { "MMRCurrentAuctionAveragePrice",
                         "MMRCurrentAuctionCleanPrice",
                         "MMRCurrentRetailAveragePrice",
                         "MMRCurrentRetailCleanPrice",
                         "MMRAcquisitionAuctionCleanPrice",
                         "MMRAcquisitonRetailCleanPrice",
                         "VehYear"
                        }

for d in corr_delete_candidate:
    train.drop(d, inplace=True, axis=1)
    test.drop(d, inplace=True, axis=1)
    
corrMatt = train.corr()
mask = np.array(corrMatt)
mask[np.tril_indices_from(mask)] = False
fig,ax= plt.subplots()
fig.set_size_inches(120,60)
sns.heatmap(corrMatt, cmap="Greens", mask=mask,vmax=.8, square=True,annot=True)

In [ ]:
train[categorical_features].describe()

## NaN handling

In [ ]:
print(test.isnull().sum())

Медіаною заповннюємо чисельні колонки, модою - категоріальні.

In [ ]:
train.fillna((train.median()), inplace=True)
test.fillna((train.median()), inplace=True)

In [ ]:
train.fillna((train.mode()), inplace=True)
test.fillna((train.mode()), inplace=True)

Утворюємо нові категорії для тих колонок, в яких надто багато missing values, проте недостатнь багато, щоб сказати, що дані не валідні.

In [ ]:
nan_new_cat_candidate = { "PRIMEUNIT", "AUCGUART", "WheelType" }

for c in nan_new_cat_candidate:
    train[c] = train[c].cat.add_categories('Unkown')
    train[c] = train[c].fillna(value="Unkown", inplace=False)
    test[c] = test[c].cat.add_categories('Unkown')
    test[c] = test[c].fillna(value="Unkown", inplace=False)


In [ ]:
print(train.isnull().sum())

In [ ]:
print(test.isnull().sum())

In [ ]:
train[categorical_features].describe()

In [ ]:
test[categorical_features].describe()

Видаляємо рядки в котрих у колонці Transmission NaN

In [ ]:
train["Transmission"].value_counts(dropna=False)

In [ ]:
train["Transmission"].replace("Manual", "MANUAL", inplace=True)

## Label Encoding for cats

In [ ]:
label_enc_candidate = { "AUCGUART", "WheelType" }

for c in label_enc_candidate:
    train[c] = train[c].cat.codes
    test[c] = test[c].cat.codes
    categorical_features.remove(c)

In [ ]:
train[categorical_features].describe()

In [ ]:
one_hot_enc_candidates = { "Nationality", "TopThreeAmericanName", 'Size', 'Auction', 
                          'IsOnlineSale', 'Transmission', 'PRIMEUNIT'
                         }

train = pd.get_dummies(train, columns=one_hot_enc_candidates, prefix = one_hot_enc_candidates)
test = pd.get_dummies(test, columns=one_hot_enc_candidates, prefix = one_hot_enc_candidates)

In [ ]:
train.dtypes

## Balance data(over-sampling)

In [ ]:
train["IsBadBuy"].value_counts(dropna=False)

In [ ]:
count_class_0, count_class_1 = train.IsBadBuy.value_counts()

df_class_0 = train[train['IsBadBuy'] == 0]
df_class_1 = train[train['IsBadBuy'] == 1]

df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

print(df_test_over.IsBadBuy.value_counts())

df_test_over.IsBadBuy.value_counts().plot(kind='bar', title='Count (target)');

In [ ]:
train = df_test_over

# Logistical regr with cross-val and ROC

Перший варіант тренування моделі за допомогою LogisticRegression 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [ ]:
num_features = train.select_dtypes(include = ['float64', 'int64', 'uint8', 'int8']).columns

In [ ]:
X_test = test[num_features]
X_train, X_val, y_train, y_val = train_test_split(train[num_features], train["IsBadBuy"], test_size=0.33, random_state=42)

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_val = sc.transform(X_val)
X_test = sc.transform(X_test)

In [ ]:
lr = LogisticRegression(solver="newton-cg", random_state=0).fit(X_train, y_train)
roc_auc_score(y_val, lr.predict_proba(X_val)[:, 1])

In [ ]:
sub_1 = pd.DataFrame(test['RefId'])
sub_1['IsBadBuy'] = lr.predict_proba(X_test)[:, 1]

In [ ]:
sub_1.to_csv('log_reg.csv', index = False)

# LGBMClassifier

Фінальний варіант тренування моделі за допомогою LGBMClassifier 

In [ ]:
from lightgbm import LGBMClassifier


In [ ]:
train_labels = train["IsBadBuy"]
train=train.drop("IsBadBuy",1)

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'
clf = LGBMClassifier()
clf.fit(train, train_labels)
predictions = clf.predict_proba(test)[:, 1]
submit = test[['RefId']]
submit['IsBadBuy'] = predictions

In [ ]:
submit.to_csv('lightgbm_baseline.csv', index = False)